<a href="https://colab.research.google.com/github/jdeiros/soa-2020/blob/master/Deiros_Jeronimo_ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('ejercicio 3')

ejercicio 3
